In [1]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.agents.agent_types import AgentType

1. __Producing well-formatted research reports__


Lets test each functionality of a potential research agent individually and then join these functionalities together at the end gradually.

In [2]:
# Import things that are needed generically
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [3]:
llm = ChatOpenAI(temperature=0)

In [4]:
# Load the tool configs that are needed.
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events"
        # coroutine= ... <- you can specify an async method if desired as well
    ),
]

/usr/local/python/3.10.8/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [18]:
agent.run("Research the current proven specific use cases for LLM agents and write a list of the actual tasks these agents can perform.")



> Entering new AgentExecutor chain...
I should search for recent articles or research papers on LLM agents to find the most up-to-date information on their specific use cases and tasks.
Action: Search
Action Input: "current proven specific use cases for LLM agents"
Observation: Welcome back to the second part of our series about identifying use cases for Large Language Models (LLMs) in the enterprise. In part 1, we delved into five ...
Thought:This article seems to provide information on the specific use cases for LLM agents. I should continue reading to gather more details.
Action: Search
Action Input: Continue reading the article
Observation: It's an easy way to distinguish actual readers from skimmers, which enables them to qualify visitors for ads.
Thought:This article does not provide the specific tasks that LLM agents can perform. I should continue searching for more sources.
Action: Search
Action Input: "tasks performed by LLM agents"
Observation: Current Uses of LLM Agents On

'The specific tasks that LLM agents can perform include answering emails, scheduling appointments, conducting conversations, completing tasks, reasoning, and demonstrating autonomous behavior.'

In [13]:
agent = initialize_agent(tools,llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [20]:
load_dotenv()
from langchain.document_loaders import BrowserlessLoader

loader = BrowserlessLoader(
    api_token=os.environ["BROWSERLESS_API_TOKEN"],
    urls=[
        "https://www.browserless.io/blog/2023/07/18/langchain-browserless/",
    ]
)

documents = loader.load()

print(documents[0].page_content)

🎉 The 2023 Browser Conference is coming! Sign-up here. 🎉
Browserless
Browserless now runs in LangChain
Joel Griffith
July 18, 2023

We’ve begun the process of integrating Browserless with the popular LangChain AI library, starting with Browserless’ REST APIs. As of today, our content API is now supported as a LangChain document loader. Using Browserless to get the contents of a webpage for ingestion into LangChain’s AI modules is as easy as calling from langchain.document_loaders import BrowserlessLoader

What this means

The previous canonical way to get the contents of webpages in LangChain was the WebBaseLoader module. This module uses the requests library to make HTTP requests to the target URL. This is a perfectly valid way to get the contents of a webpage, but it has some drawbacks:

It doesn’t execute JavaScript, so it can’t get the contents of a webpage that is dynamically generated by JavaScript.
It’s prone to encoding issues if Python is expecting a different encoding than th

In [21]:
load_dotenv()
# metaphor search

from metaphor_python import Metaphor

client = Metaphor(api_key=os.environ["METAPHOR_API_KEY"])

In [22]:
from langchain.agents import tool
from typing import List

In [23]:
@tool
def search(query: str):
    """Call search engine with a query."""
    return client.search(query, use_autoprompt=True, num_results=5)

@tool
def get_contents(ids: List[str]):
    """Get contents of a webpage.
    
    The ids passed in should be a list of ids as fetched from `search`.
    """
    return client.get_contents(ids)

@tool
def find_similar(url: str):
    """Get search results similar to a given URL.
    
    The url passed in should be a URL returned from `search`
    """
    return client.find_similar(url, num_results=5)

In [24]:
tools = [search, get_contents, find_similar]

In [25]:
llm = ChatOpenAI(temperature=0)

In [26]:
from langchain.agents import OpenAIFunctionsAgent
from langchain.schema import SystemMessage
system_message = SystemMessage(content="You are a web researcher who uses search engines to look up information.")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

In [28]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [29]:
agent_executor.run("Research 5 relevant papers about using machine learning to augment human intelligence.")



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': 'machine learning to augment human intelligence papers'}`


Title: Learning to Complement Humans
URL: https://arxiv.org/abs/2005.00582v1
ID: BG23MJc9WT0uNA0Clfi14w
Score: 0.20907114446163177
Published Date: 2020-05-01
Author: Wilder; Bryan; Horvitz; Eric; Kamar; Ece
Extract: None

Title: Learning to Complement Humans
URL: https://arxiv.org/abs/2005.00582
ID: DudqIh0uSTZG_9IEgZrOLw
Score: 0.2060324251651764
Published Date: 2020-05-01
Author: Wilder; Bryan; Horvitz; Eric; Kamar; Ece
Extract: None

Title: [2005.00582] Learning to Complement Humans
URL: https://arxiv.org/pdf/2005.00582.pdf
ID: BYMb2c3Zmf2WMiePHhl5ZQ
Score: 0.20324468612670898
Published Date: None
Author: None
Extract: None

Title: Accelerating science with human versus alien artificial intelligences
URL: https://arxiv.org/abs/2104.05188
ID: 15QIvYbLun8L7IRc0nXq0g
Score: 0.19190381467342377
Published Date: 2021-04-12
Author: Sourati; Jamshid; Evans;

'Here are 5 relevant papers about using machine learning to augment human intelligence:\n\n1. "Learning to Complement Humans" by Bryan Wilder, Eric Horvitz, and Ece Kamar. [Link to paper](https://arxiv.org/abs/2005.00582v1)\n\n2. "Accelerating science with human versus alien artificial intelligences" by Jamshid Sourati and James Evans. [Link to paper](https://arxiv.org/abs/2104.05188)\n\n3. "Accelerating science with human-aware artificial intelligence" by Jamshid Sourati and James Evans. [Link to paper](https://arxiv.org/abs/2306.01495)\n\nPlease note that the other papers found do not have available extracts or published dates.'

One obvious workflow that I could set up is to have the agent read specific URLs that I know might contain the content I am looking for and then, try to answer some question or solve some problem.

Research Agent 
copied from 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from research_agent import *

In [3]:
# 3. Create langchain agent with the tools above
tools = [
    Tool(
        name="Search",
        func=search,
        description="useful for when you need to answer questions about current events, data. You should ask targeted questions"
    ),
    ScrapeWebsiteTool(),
]

system_message = SystemMessage(
    content="""You are a world class researcher, who can do detailed research on any topic and produce facts based results; 
            you do not make things up, you will try as hard as possible to gather facts & data to back up the research
            
            Please make sure you complete the objective above with the following rules:
            1/ You should do enough research to gather as much information as possible about the objective
            2/ If there are url of relevant links & articles, you will scrape it to gather more information
            3/ After scraping & search, you should think "is there any new things i should search & scraping based on the data I collected to increase research quality?" If answer is yes, continue; But don't do this more than 3 iteratins
            4/ You should not make things up, you should only write facts & data that you have gathered
            5/ In the final output, You should include all reference data & links to back up your research; You should include all reference data & links to back up your research
            6/ In the final output, You should include all reference data & links to back up your research; You should include all reference data & links to back up your research"""
)

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
    "system_message": system_message,
}

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k-0613")
memory = ConversationSummaryBufferMemory(
    memory_key="memory", return_messages=True, llm=llm, max_token_limit=1000)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
    memory=memory,
)

In [4]:
query = "What is the latest prompt engineering technique to get the most out of a large language model?"
agent({"input": query})



> Entering new AgentExecutor chain...

Invoking: `Search` with `latest prompt engineering technique to get the most out of a large language model`


{"message":"Unauthorized.","statusCode":403}
{"message":"Unauthorized.","statusCode":403}I apologize, but I am unable to perform a search at the moment. However, I can provide you with some general information about prompt engineering techniques for large language models.

Prompt engineering refers to the process of designing and formulating effective prompts or instructions to elicit desired responses from a language model. It involves crafting specific input text that guides the model to generate the desired output.

Here are a few commonly used prompt engineering techniques to get the most out of a large language model:

1. Contextual Prompts: Providing relevant context in the prompt can help the model generate more accurate and contextually appropriate responses. Including specific details, background information, or examples can gui

{'input': 'What is the latest prompt engineering technique to get the most out of a large language model?',
 'memory': [HumanMessage(content='What is the latest prompt engineering technique to get the most out of a large language model?', additional_kwargs={}, example=False),
  AIMessage(content="I apologize, but I am unable to perform a search at the moment. However, I can provide you with some general information about prompt engineering techniques for large language models.\n\nPrompt engineering refers to the process of designing and formulating effective prompts or instructions to elicit desired responses from a language model. It involves crafting specific input text that guides the model to generate the desired output.\n\nHere are a few commonly used prompt engineering techniques to get the most out of a large language model:\n\n1. Contextual Prompts: Providing relevant context in the prompt can help the model generate more accurate and contextually appropriate responses. Includi

Producing well-formatted research reports:
- ~~Researching basic questions and problems~~
- Testing hypothesis on different problems
- Designing and executing experiments
- Doing Literature review on a topic of interest
- Gathering data and data sources for a particular problem or research topic
- Creating exercises and challenges
- Creating novel flashcards to challenge what I know and help me practice for better encoding
- Summarizing papers and web-articles
- Creating curriculums for learning a tough subject
- Evaluating performance when learning something new and tracking my progress
- Document Q&A - Chating with papers, research documents and notes
- Brainstorming tool for learning, research and content

In [ ]:
def call_researcher_agent(query):
    return agent({"input": query})


call_researcher_agent("Research the validity of the following hypothesis: ADHD is mostly caused by low dopaminergic levels in the brain.")



> Entering new AgentExecutor chain...

Invoking: `Search` with `ADHD and low dopaminergic levels in the brain`


{"message":"Unauthorized.","statusCode":403}
{"message":"Unauthorized.","statusCode":403}I apologize, but I am currently unable to perform a search to gather information on the validity of the hypothesis that ADHD is mostly caused by low dopaminergic levels in the brain. However, I can provide you with some general information about the relationship between ADHD and dopamine levels.

Attention-Deficit/Hyperactivity Disorder (ADHD) is a neurodevelopmental disorder characterized by symptoms of inattention, hyperactivity, and impulsivity. The exact cause of ADHD is not fully understood, and it is likely to be influenced by a combination of genetic, environmental, and neurological factors.

Dopamine is a neurotransmitter that plays a crucial role in various brain functions, including attention, motivation, and reward processing. It is believed that imbalances in dopamine neuro

{'input': 'Research the validity of the following hypothesis: ADHD is mostly caused by low dopaminergic levels in the brain.',
 'memory': [HumanMessage(content='What is the latest prompt engineering technique to get the most out of a large language model?', additional_kwargs={}, example=False),
  AIMessage(content="I apologize, but I am unable to perform a search at the moment. However, I can provide you with some general information about prompt engineering techniques for large language models.\n\nPrompt engineering refers to the process of designing and formulating effective prompts or instructions to elicit desired responses from a language model. It involves crafting specific input text that guides the model to generate the desired output.\n\nHere are a few commonly used prompt engineering techniques to get the most out of a large language model:\n\n1. Contextual Prompts: Providing relevant context in the prompt can help the model generate more accurate and contextually appropriate